In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pulp
import time

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patches as mpatches

import seaborn as sns
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable

from shapely.geometry import LineString

import warnings
warnings.filterwarnings('ignore')

In [2]:
# pulp 버전 확인
print(pulp.__version__)

3.0.2


#### 5km 이내 데이터 생성

##### 데이터 로드

In [5]:
# 100m 격자 데이터 로드

grid_bukgu= gpd.read_file('input_data/gis/100M_부산광역시/100M_부산광역시 북구_202410',encoding="UTF-8")
grid_bukgu = grid_bukgu.loc[grid_bukgu['val']>0].reset_index(drop=True)

grid_sasang= gpd.read_file('input_data/gis/100M_부산광역시/100M_부산광역시 사상구_202410',encoding="UTF-8")
grid_sasang = grid_sasang.loc[grid_sasang['val']>0].reset_index(drop=True)

grid_saha= gpd.read_file('input_data/gis/100M_부산광역시/100M_부산광역시 사하구_202410',encoding="UTF-8")
grid_saha = grid_saha.loc[grid_saha['val']>0].reset_index(drop=True)

grid = pd.concat([grid_bukgu,grid_sasang,grid_saha])

In [6]:
# WGS84 (EPSG:4326) 좌표계로 변환
grid = grid.to_crs(epsg=4326)

# grid["centroid"] = grid["geometry"].centroid
grid.drop_duplicates(subset=['gid'], keep='first', inplace=True)
grid = grid.reset_index(drop=True)

grid["centroid"] = grid["geometry"].centroid
grid["geometry_2"] = grid["geometry"]
grid['geometry'] = grid['centroid']


# grid.to_file('grid.shp', encoding='utf-8')

# 행정동 데이터 로드
path = "input_data/행정동/HJD"
# .shp 파일을 읽어옵니다
hjd = gpd.read_file(path, encoding='cp949')
hjd = hjd.set_crs(epsg=5179, allow_override=True)
hjd = hjd.to_crs(epsg=4326)


# 행정동 데이터 로드
path = "input_data/행정동/EMD"
# .shp 파일을 읽어옵니다
emd = gpd.read_file(path, encoding='cp949')
emd = emd.set_crs(epsg=5179, allow_override=True)
emd = emd.to_crs(epsg=4326)


# 공간 조인을 수행하여 gdf의 center가 emd_data의 geometry에 포함되는지 확인
gdf = gpd.sjoin(grid, emd[['EMD_CD','EMD_KOR_NM', 'geometry']], how='left', predicate='within')
len(gdf)

# 2020 : 장림동 , 2869 : 다대동으로 지정 ( 네이버 지도 참고)
none_rows = gdf[gdf['EMD_KOR_NM'].isna()]
# gdf[gdf.isna().any(axis=1)]

# 수동으로 EMD_KOR_NM 값을 수정
gdf.loc[2009, 'EMD_KOR_NM'] = '장림동'
gdf.loc[2857, 'EMD_KOR_NM'] = '다대동'

# 수동으로 EMD_CD 값을 수정
gdf.loc[2009, 'EMD_CD'] = 26380105
gdf.loc[2857, 'EMD_CD'] = 26380106


In [168]:
# gdf_geo = gdf.drop(['centroid','index_right','geometry','centroid'], axis =1)
# gdf_geo = gdf_geo.rename(columns={
#                         'geometry_2': 'geometry',
#                     })
# gdf_geo.to_file('input_data/gdf/gdf.shp', driver='ESRI Shapefile', encoding = 'utf-8')

##### 중복쌍 없이 생성

In [6]:
from itertools import combinations
from tqdm import tqdm

# centroid에서 경도(longitude)와 위도(latitude) 추출
gdf['longitude'] = gdf['centroid'].x
gdf['latitude'] = gdf['centroid'].y

# Generate all combinations of rows where i < j
combinations_data = []
for (i, row_i), (j, row_j) in tqdm(combinations(gdf.iterrows(), 2), total=len(gdf) * (len(gdf) - 1) // 2):
    combinations_data.append({
        'start_gid': row_i['gid'],
        'start_EMD_CD': row_i['EMD_CD'],
        'start_EMD_KOR_NM': row_i['EMD_KOR_NM'],
        'start_longitude': row_i['longitude'],
        'start_latitude': row_i['latitude'],
        'end_gid': row_j['gid'],
        'end_EMD_CD': row_j['EMD_CD'],
        'end_EMD_KOR_NM': row_j['EMD_KOR_NM'],
        'end_longitude': row_j['longitude'],
        'end_latitude': row_j['latitude'],
    })
    
    
# Create a new DataFrame from the combinations
df_combinations = pd.DataFrame(combinations_data)

print(f"생성된 총 쌍의 수: {len(df_combinations)}")

100%|██████████| 4444671/4444671 [00:43<00:00, 103340.05it/s]


생성된 총 쌍의 수: 4444671


##### 5km 이내

In [7]:
warnings.filterwarnings('ignore')

###################################################
def calculate_straight_distance(lat1, lon1, lat2, lon2):

    km_constant = 3959* 1.609344
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    km = km_constant * c
    return km

# 직선 거리 계산
df_combinations['straight_distance_km'] = df_combinations.apply(
    lambda row: calculate_straight_distance(
        row['start_latitude'], 
        row['start_longitude'], 
        row['end_latitude'], 
        row['end_longitude']
    ), 
    axis=1
)

# 5km 이하인 것만 필터링
df_filtered = df_combinations[df_combinations['straight_distance_km'] <= 5]

# 결과 확인
print(f"원본 데이터 개수: {len(df_combinations)}")
print(f"5km 이하 필터링 후 데이터 개수: {len(df_filtered)}")
df_filtered.head()


원본 데이터 개수: 4444671
5km 이하 필터링 후 데이터 개수: 1636281


,start_gid,start_EMD_CD,start_EMD_KOR_NM,start_longitude,start_latitude,end_gid,end_EMD_CD,end_EMD_KOR_NM,end_longitude,end_latitude,straight_distance_km
1,마라361908,26320105,구포동,128.995688,35.203767,마라384924,26320104,덕천동,129.021216,35.217875,2.800106
2,마라361908,26320105,구포동,128.995688,35.203767,마라378940,26320102,화명동,129.014893,35.232379,3.628767
3,마라361908,26320105,구포동,128.995688,35.203767,마라392918,26320103,만덕동,129.029901,35.212355,3.252027
4,마라361908,26320105,구포동,128.995688,35.203767,마라364905,26320105,구포동,128.998933,35.201022,0.424399
5,마라361908,26320105,구포동,128.995688,35.203767,마라375927,26320102,화명동,129.011380,35.220703,2.362082


In [34]:
import pickle

# Save the DataFrame to a CSV file
csv_file_path = 'output_data/5km_distance/distance_df.csv'
df_filtered.to_csv(csv_file_path, index=False, encoding='cp949')

#### osrm 거리

In [15]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

###################################################
def calculate_straight_distance(lat1, lon1, lat2, lon2):

    km_constant = 3959* 1.609344
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    km = km_constant * c
    return km
    

#############
# OSRM base #
#############

def get_res_osrm(point): # lat,lon,lat.lon

   status = 'defined'

   session = requests.Session()
   retry = Retry(connect=10, backoff_factor=1)
   adapter = HTTPAdapter(max_retries=retry)
   session.mount('http://', adapter)
   session.mount('https://', adapter)

   overview = '?overview=full'
   # step = '?steps=true'
   loc = f"{point[0]},{point[1]};{point[2]},{point[3]}" # lon, lat, lon, lat
   url = "http://localhost:5001/route/v1/car/"
   # url = 'http://router.project-osrm.org/route/v1/driving/'
   r = session.get(url + loc + overview) 
   
   if r.status_code!= 200:
      
      status = 'undefined'
      
      # distance    
      distance = calculate_straight_distance(point[1], point[0], point[3], point[2]) * 1000
      
      # route
      route = [[point[0], point[1]], [point[2], point[3]]]

      # duration & timestamp
      speed_km = 50#km
      speed = (speed_km * 1000/60)      
      duration = distance/speed
      
      timestamp = [0, duration] 

      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
   
      return result, status

   res = r.json()   
   return res, status


def get_res_osrm_5000(point, port): # lat,lon,lat.lon

   status = 'defined'

   session = requests.Session()
   retry = retry = Retry(connect=10, backoff_factor=1)
   adapter = HTTPAdapter(max_retries=retry)
   session.mount('http://', adapter)
   session.mount('https://', adapter)

   overview = '?overview=full'
   # step = '?steps=true'
   loc = f"{point[0]},{point[1]};{point[2]},{point[3]}" # lon, lat, lon, lat
   url = f"http://localhost:{port}/route/v1/car/"
   # url = 'http://router.project-osrm.org/route/v1/driving/'
   r = session.get(url + loc + overview) 
   
   if r.status_code!= 200:
      
      status = 'undefined'
      
      # distance    
      distance = calculate_straight_distance(point[1], point[0], point[3], point[2]) * 1000
      
      # route
      route = [[point[0], point[1]], [point[2], point[3]]]

      # duration & timestamp
      speed_km = 50#km
      speed = (speed_km * 1000/60)      
      duration = distance/speed
      
      timestamp = [0, duration] 

      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
   
      return result, status

   res = r.json()   
   return res, status

##############################
# Extract duration, distance #
##############################
def extract_duration_distance(res):
       
   # duration = res['routes'][0]['duration']
   duration = res['routes'][0]['duration']/(60)
   distance = res['routes'][0]['distance']
   
   return duration, distance

osrm_ports = [5000, 5001]


########
# Port #
########
def get_current_port():
    global current_port_index
    return osrm_ports[current_port_index]

def update_port_if_needed(iteration):
    global current_port_index
    if iteration % 3 == 0 and iteration != 0:
        current_port_index = (current_port_index + 1) % len(osrm_ports)
        

###################################################

In [ ]:
import pickle

# Save function
def save_data(data, filename='df_filtered_checkpoint.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

# Main loop
save_interval = 15000  # Save every 15000 entries
start_index = 0  # Starting index

for i, entry in enumerate(tqdm(df_filtered[start_index:].iterrows(), initial=start_index, total=len(df_filtered))):
    index, row = entry
    point = (
        row['start_longitude'], 
        row['start_latitude'], 
        row['end_longitude'], 
        row['end_latitude']
    )  # lon,lat,lon,lat
    
    # Get OSRM response
    res, status = get_res_osrm_5000(point, 5000)
    duration, distance = extract_duration_distance(res)
    
    # Update the dataframe
    df_filtered.at[index, 'distance'] = distance
    df_filtered.at[index, 'duration'] = duration

    # Save at intervals
    if (i + start_index + 1) % save_interval == 0:
        save_data(df_filtered)

##### 데이터 확인 - r5r로 돌린거

In [ ]:
# import pandas as pd
# import numpy as np
# import re

# def fill_nan_values(second_df, first_df):
#     # NaN 값 채우기
#     # 1. start_gid 기준으로 첫 번째 데이터셋에서 EMD_CD와 EMD_KOR_NM 찾기
#     for index, row in second_df.iterrows():
#         if pd.isna(row['start_EMD_CD']) or row['start_EMD_CD'] == 'NaN':
#             start_gid = row['start_gid']
#             matching_row = first_df[first_df['gid'] == start_gid]
            
#             if not matching_row.empty:
#                 emd_cd = matching_row.iloc[0]['EMD_CD']
#                 emd_kor_nm = matching_row.iloc[0]['EMD_KOR_NM']
                
#                 second_df.at[index, 'start_EMD_CD'] = emd_cd
#                 second_df.at[index, 'start_EMD_KOR_NM'] = emd_kor_nm
    
#     # 2. end_gid 기준으로 첫 번째 데이터셋에서 EMD_CD와 EMD_KOR_NM 찾기
#     for index, row in second_df.iterrows():
#         if pd.isna(row['end_EMD_CD']) or row['end_EMD_CD'] == 'NaN':
#             end_gid = row['end_gid']
#             matching_row = first_df[first_df['gid'] == end_gid]
            
#             if not matching_row.empty:
#                 emd_cd = matching_row.iloc[0]['EMD_CD']
#                 emd_kor_nm = matching_row.iloc[0]['EMD_KOR_NM']
                
#                 second_df.at[index, 'end_EMD_CD'] = emd_cd
#                 second_df.at[index, 'end_EMD_KOR_NM'] = emd_kor_nm
    
#     return second_df


# first_df = gdf

# # 두 번째 데이터셋 (NaN 값을 채울 데이터) 읽기
# second_df = merge_df

# # NaN 값 채우기
# filled_df = fill_nan_values(second_df, first_df)
# print("NaN 값 채우기 완료")


In [8]:
### 데이터 로드
distance_df = pd.read_csv('output_data/5km_distance/distance_df_.csv', encoding='cp949')
distance_df_na = pd.read_csv('output_data/5km_distance/TCD_final.csv')


```
    7000행 정도 결과가 나오지 않음    
```

In [9]:
merge_cols = [
    'start_gid', 'start_EMD_CD', 'start_EMD_KOR_NM', 'start_longitude', 'start_latitude', 
    'end_gid', 'end_EMD_CD', 'end_EMD_KOR_NM', 'end_longitude', 'end_latitude', 'best_total_duration'
]

distance_df_na = distance_df_na[merge_cols]
distance_df_na.rename(columns = {'best_total_duration':'travel_time_p50'}, inplace = True)

# gid만으로 매칭
distance_df['start_end_key'] = distance_df['start_gid'] + '_' + distance_df['end_gid']
distance_df_na['start_end_key'] = distance_df_na['start_gid'] + '_' + distance_df_na['end_gid']


# 매핑 딕셔너리 생성
travel_time_map = dict(zip(distance_df_na['start_end_key'], distance_df_na['travel_time_p50']))

# distance_df의 travel_time_p50 채우기
mask_1 = distance_df['travel_time_p50'].isna()
# mask_2 = distance_df['travel_time_p50'] == 0

distance_df.loc[mask_1, 'travel_time_p50'] = distance_df.loc[mask_1, 'start_end_key'].map(travel_time_map)

# distance_df.loc[mask_2, 'travel_time_p50'] = distance_df.loc[mask_2, 'match_key'].map(travel_time_map)

# 매칭 컬럼 제거 (필요한 경우)
distance_df = distance_df.drop('start_end_key', axis=1)

In [139]:
# Save the DataFrame to a CSV file
csv_file_path = 'output_data/5km_distance/distance_df_final.csv'
distance_df.to_csv(csv_file_path, index=False, encoding='cp949')

In [62]:
# dinstancd_df_0_na = distance_df[(distance_df['travel_time_p50'] == 0) | (distance_df.isna().any(axis = 1))]

# # Save the DataFrame to a CSV file
# csv_file_path = 'output_data/5km_distance/dinstancd_df_0_na.csv'
# dinstancd_df_0_na.to_csv(csv_file_path, index=False, encoding='cp949')